In [46]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup

In [44]:
def get_discursos_urls(base_url, keyword):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, "html.parser")
    div_selector = "html body div#jm-allpage.nofluid div#jm-mainpage div#jm-mainpage-in div#jm-main.lcr.scheme1.nocolumns.clearfix div#jm-maincontent main.home-special.home-mid div.container section div.row.row-extra.row-news.row-clear-4 div.blog div.contentboxes div.box.col-sm-6.col-md-3 div.item"
    divs = soup.select(div_selector)

    discursos_urls = [
        f"{base_url}{a.get('href')}"
        for div in divs
        for a in div.find_all("a")
        if keyword in a.get("href").lower()
    ]

    return discursos_urls

base_url = "https://www.casarosada.gob.ar/informacion/discursos/"
keyword = "milei"
discursos_urls = get_discursos_urls(base_url, keyword)

In [47]:
# Define a dictionary to map Spanish month names to numbers
month_map = {
    "enero": "01",
    "febrero": "02",
    "marzo": "03",
    "abril": "04",
    "mayo": "05",
    "junio": "06",
    "julio": "07",
    "agosto": "08",
    "septiembre": "09",
    "octubre": "10",
    "noviembre": "11",
    "diciembre": "12",
}


def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup


def get_title(soup):
    return soup.find("h2").text if soup.find("h2") else np.nan


def get_article_content(soup):
    return "".join(
        [p.text for p in soup.find("article").find_all("p") if p.find("strong") is None]
        if soup.find("article")
        else np.nan
    )


def get_date(soup):
    date = (
        soup.find("time").text.replace("\r", "").replace("\n", "")
        if soup.find("time")
        else np.nan
    )
    # Remove the day of the week from the date string
    date = " ".join(date.split()[1:])
    # Replace the Spanish month name with its corresponding number
    for month_name, month_number in month_map.items():
        date = date.replace(month_name, month_number)
    # Convert the date string to a date object and format it in 'YYYY-MM-DD'
    return datetime.datetime.strptime(date, "%d de %m de %Y").strftime("%Y-%m-%d")


def create_dataframe(discursos_urls):
    """
    Creates a DataFrame from the list of URLs.
    """
    data = {"title": [], "content": [], "date": [], "url": []}
    for url in discursos_urls:
        soup = get_content(url)
        title = get_title(soup)
        content = get_article_content(soup)
        date = get_date(soup)
        data["title"].append(title)
        data["content"].append(content)
        data["date"].append(date)
        data["url"].append(url)
    return pd.DataFrame(data)


df = create_dataframe(discursos_urls)
df

,title,content,date,url
0,Discurso completo del Presidente Javier Milei ...,"En primer lugar, deseo dar las gracias por est...",2024-06-24,https://www.casarosada.gob.ar/informacion/disc...
1,"Palabras del Presidente de la Nación, Javier M...","Buenas tardes y, obviamente, si estuviera en u...",2024-06-22,https://www.casarosada.gob.ar/informacion/disc...
2,"Palabras del Presidente de la Nación, Javier M...","Buenas noches. En primer lugar, quiero dar las...",2024-06-21,https://www.casarosada.gob.ar/informacion/disc...
3,"Palabras del Presidente de la Nación, Javier M...","Buenas tardes. Muchísimas gracias, presidente ...",2024-06-21,https://www.casarosada.gob.ar/informacion/disc...
4,"Palabras del Presidente de la Nación, Javier M...","Argentinos, hoy conmemoramos el paso a la inmo...",2024-06-20,https://www.casarosada.gob.ar/informacion/disc...
5,"Palabras del Presidente de la Nación, Javier M...",Buenas tardes a todos. Es un honor para mí y p...,2024-06-15,https://www.casarosada.gob.ar/informacion/disc...
6,Discurso del Presidente Javier Milei en la con...,Javier Milei: En primer lugar quiero agradecer...,2024-06-12,https://www.casarosada.gob.ar/informacion/disc...
7,"Palabras del Presidente de la Nación, Javier M...","Buenos días. Muchísimas gracias, Esteban, por ...",2024-06-12,https://www.casarosada.gob.ar/informacion/disc...
8,"Palabras del Presidente de la Nación, Javier M...",PRESIDENTE. - Muchas gracias a todos los prese...,2024-06-06,https://www.casarosada.gob.ar/informacion/disc...
9,"Palabras del Presidente de la Nación, Javier M...","Buenas tardes. Bueno, en primer lugar, muchas ...",2024-06-05,https://www.casarosada.gob.ar/informacion/disc...
